In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os

with open("dataset.json","r") as f:
    datafile=json.load(f)
for game in datafile:
    if "genres" not in datafile[game].keys():
        datafile[game]["genres"]=[]
    if "languages" not in datafile[game].keys():
        datafile[game]["languages"]=[]
df= pd.DataFrame.from_dict(datafile)
print(df.head())

                                                     2694490  \
title                                        Path of Exile 2   
ranks      {'rank_LK': 1, 'rank_JP': 1, 'rank_US': 1, 'ra...   
languages  [English, French, German, Spanish - Spain, Jap...   
genres       [Action, Adventure, RPG, Massively Multiplayer]   
url        [https://store.steampowered.com/app/2694490, h...   

                                                         730  \
title                                       Counter-Strike 2   
ranks      {'rank_LK': 2, 'rank_VN': 1, 'rank_CN': 1, 'ra...   
languages  [English, Czech, Danish, Dutch, Finnish, Frenc...   
genres                                             [Shooter]   
url        [https://store.steampowered.com/app/730, https...   

                                                     2767030  \
title                                          Marvel Rivals   
ranks      {'rank_LK': 3, 'rank_VN': 2, 'rank_CN': 6, 'ra...   
languages  [English, French, Italian, 

In [37]:
languages=[]
for app in df:
    if "languages" in df[app].keys():
        if type(df[app]["languages"]) is list:
            languages=languages+(df[app]["languages"])
languages=list(set(languages))
print(languages)


['Sotho', 'Cherokee', 'Swahili', 'Vietnamese', 'Georgian', 'Hindi', 'Lithuanian', 'Punjabi (Shahmukhi)', 'Catalan', 'Swedish', 'Konkani', 'Malayalam', 'Bulgarian', 'Azerbaijani', 'Tajik', 'Wolof', 'Traditional Chinese', "K'iche'", 'Khmer', 'Greek', 'Belarusian', 'Gujarati', '#lang_#lang_russian*#lang_full_audio', 'Kyrgyz', 'Quechua', 'Uyghur', 'Sindhi', 'Dari', 'Danish', 'German', 'Sorani', 'Icelandic', 'Valencian', 'Punjabi (Gurmukhi)', 'Assamese', 'Maori', 'Armenian', 'Czech', 'Scots', 'Portuguese - Portugal', 'Italian', 'Croatian', 'Urdu', 'Filipino', 'Amharic', 'Dutch', 'Welsh', 'Turkish', 'Maltese', 'Igbo', 'Uzbek', 'Bangla', 'Bosnian', 'Zulu', 'Romanian', 'Norwegian', 'Spanish - Latin America', 'Odia', 'Yoruba', 'Slovak', 'Korean', 'Serbian', 'French', 'Kazakh', '#lang_german;', 'Xhosa', 'Tatar', 'Arabic', 'Kannada', 'Telugu', 'Irish', 'Galician', 'Ukrainian', 'Spanish - Spain', 'Portuguese - Brazil', 'Basque', 'Tswana', 'Macedonian', 'Luxembourgish', 'Indonesian', 'Thai', 'Slove

In [38]:

genres=[]
for app in df:
    if "genres" in df[app].keys():
        if type(df[app]["genres"]) is list:
            genres+=df[app]["genres"]
genres=list(set(genres))
print(genres)


['Board Games', 'Shooter', 'Puzzle', 'Arcade', 'Casual', 'Indie', 'Family', 'Massively Multiplayer', 'Action', 'Strategy', 'RPG', 'Sports', 'Card', 'Educational', 'Racing', 'Simulation', 'Adventure', 'Platformer', 'Fighting']


In [39]:
import numpy as np
#Avaliação entre idioma e popularidade no país
country='BR' #Definir qual país pesquisar aqui
flat_data_genre=[]
for id,data in datafile.items():
    rank=data["ranks"]["rank_"+country]
    if rank==-1:
        continue
    genres=data["genres"]
    for g in genres:
        flat_data_genre.append({"appid":id,"rank":rank,"genre":g})
df_genres=pd.DataFrame(flat_data_genre)
max_rank = df_genres['rank'].max()
df_genres["rank"]=np.log(max_rank - df_genres['rank']+1) #inverte o peso dos ranks, utiliza log para nivelar jogos em ranks parecidos
sum_rank_genres = df_genres.groupby('genre')['rank'].sum()

# Print the results
print(sum_rank_genres.sort_values(ascending=False).head(20))

genre
Action                   10089.852955
Indie                     9879.484676
Adventure                 8064.477492
Simulation                5570.891090
RPG                       4631.826094
Strategy                  4281.051042
Casual                    4146.974798
Shooter                   1406.788982
Massively Multiplayer     1042.143578
Sports                     854.406366
Puzzle                     839.358693
Racing                     833.332507
Platformer                 779.485440
Arcade                     470.974207
Fighting                   354.604823
Family                     248.066967
Educational                106.113020
Card                       104.894988
Board Games                 95.387730
Name: rank, dtype: float64


In [40]:
import numpy as np
#Avaliação entre gênero e popularidade no país
country='BR' #Definir qual país pesquisar aqui
flat_data_lang=[]
for id,data in datafile.items():
    rank=data["ranks"]["rank_"+country]
    if rank==-1:
        continue
    languages=data["languages"]
    for l in languages:
        flat_data_lang.append({"appid":id,"rank":rank,"language":l})

df_langs=pd.DataFrame(flat_data_lang)
max_rank = df_langs['rank'].max()
df_langs["rank"]=np.log(max_rank - df_langs['rank']+1) #inverte o peso dos ranks, utiliza log para nivelar jogos em ranks parecidos
sum_rank_langs = df_langs.groupby('language')['rank'].sum()

# Print the results
print(sum_rank_langs.sort_values(ascending=False).head(20))

language
English                    20062.540468
French                     15213.905939
German                     15171.235070
Spanish - Spain            14002.022712
Simplified Chinese         13025.831872
Japanese                   12616.957935
Russian                    12410.495459
Italian                    11651.755180
Portuguese - Brazil        11339.963322
Korean                     10148.968795
Traditional Chinese         8593.974809
Polish                      8353.549221
Turkish                     5089.215473
Spanish - Latin America     4778.806594
Portuguese - Portugal       3381.197175
Dutch                       3050.094101
Czech                       2779.611993
Ukrainian                   2229.883960
Hungarian                   1907.708726
Thai                        1891.336611
Name: rank, dtype: float64


In [43]:
#Avalia correlação entre popularidade de um jogo
#e seu preço (grande correlação= jogos caros são populares)
import numpy as np
country='US' #Definir qual país pesquisar aqui
with open(os.path.join("initial_data","currencies.json"),"r") as f:
    curr_datafile=json.load(f)
currency=curr_datafile[country]['currency']
flat_data_prices=[]
for id,data in datafile.items():
    rank=data["ranks"]["rank_"+country]
    if rank==-1:
        continue
    if "prices" in data.keys() and "price_"+currency in data["prices"].keys():
        price=data["prices"]["price_"+currency]
        if (price!=0):
            flat_data_prices.append({"appid":id,"rank":rank,"price":price})

df_prices=pd.DataFrame(flat_data_prices)
max_rank = df_prices['rank'].max()
df_prices["rank"]=np.log(max_rank - df_prices['rank']+1) #inverte os ranks,nivela-os

corr_price = df_prices[['price', 'rank']].corr()
print(corr_price)

          price      rank
price  1.000000  0.228795
rank   0.228795  1.000000


In [54]:
#Avalia correlação entre popularidade de um jogo
#e o método utilizado para conversão do seu preço em dólar para a moeda regional
#Por exemplo, se a correlação for pequena, jogos que tem seu preço reduzido
#neste mercado, em relação ao preço original em dólar, são mais populares
import numpy as np

def convert_to_dollar(source_price, source_country):
    currencies_list = open(os.path.join("initial_data", "currencies.json"), "r")
    list = json.load(currencies_list)
    if source_country in list:
        rateToDollar = list[source_country]["rateToDollar"]
        priceInDollar = source_price * rateToDollar
        return priceInDollar
    print("The selected country is not available.")
    return False


country='UA' #Definir qual país pesquisar aqui
with open(os.path.join("initial_data","currencies.json"),"r") as f:
    curr_datafile=json.load(f)
currency=curr_datafile[country]['currency']
flat_data_conv=[]
for id,data in datafile.items():
    rank=data["ranks"]["rank_"+country]
    if rank==-1:
        continue
    if "prices" in data.keys() and "price_"+currency in data["prices"].keys() and "price_USD" in data["prices"].keys():
        if (data["prices"]["price_USD"]!=0):
            conversion=convert_to_dollar(data["prices"]["price_"+currency],country)/data["prices"]["price_USD"]
            flat_data_conv.append({"appid":id,"rank":rank,"conversion":conversion})

df_conv=pd.DataFrame(flat_data_conv)
max_rank = df_conv['rank'].max()
df_conv["rank"]=np.log(max_rank - df_conv['rank']+1) #inverte os ranks, nivela-os

corr_conv = df_conv[['conversion', 'rank']].corr()
print(corr_conv)

            conversion      rank
conversion    1.000000 -0.224975
rank         -0.224975  1.000000
